In [1]:
from ipfml import processing
from ipfml import utils
from ipfml import metrics
from PIL import Image
from scipy import signal
from skimage import color
import scipy.stats as stats
import seaborn as sns
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import math

from scipy.signal import medfilt2d, wiener, cwt
import pywt

In [2]:
data_folder = "../fichiersSVD_light"

# Wavelet filter analysis on zones of Synthesis Images 

## Utils functions definition

In [3]:
def compute_images_path(dict_data):
    scene = dict_data['name']
    prefix = dict_data['prefix']
    indices = dict_data['indices']
    
    images_path = []
    for index in indices:
        path = os.path.join(data_folder, os.path.join(scene, prefix + index + ".png"))
        print(path)
        images_path.append(path)
    return images_path

In [4]:
def get_images_zones(dict_data, images_path):
    
    zones_indices = dict_data['zones']
    zones_img = []
    
    for path in images_path:
        img = Image.open(path)
        zones = processing.divide_in_blocks(img, (200, 200))
        
        zones_list = []
        
        for id_zone in zones_indices:
            zones_list.append(zones[id_zone])
            
        zones_img.append(zones_list)
        
    return zones_img

In [5]:
def display_sv_data(dict_data, zones_data, interval, _norm=False):
    
    scene_name = dict_data['name']
    image_indices = dict_data['indices']
    zones_indices = dict_data['zones']
    colors = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
    
    plt.figure(figsize=(25, 20))
    
    sv_data = []
    begin, end = interval
    for id_img, zones in enumerate(zones_data):
        
        for id_zone, zone in enumerate(zones):
            U, s, V = processing.get_LAB_L_SVD(zone)
        
            data = s[begin:end]
            
            if _norm:
                data = utils.normalize_arr(data)
                
            plt.plot(data, 
                     color=colors[id_zone], 
                     label='Zone ' + str(zones_indices[id_zone]) + ' of ' + scene_name + '_' + str(image_indices[id_img]))
            
    plt.legend(fontsize=18)
    plt.show()

In [6]:
# Useful function

def get_highest_values(arr, n):
    return np.array(arr).argsort()[-n:][::-1]

def get_lowest_values(arr, n):
    return np.array(arr).argsort()[::-1][-n:][::-1]

In [26]:
def get_sv_std(arr):
    arr = np.array(arr)
    images = [arr]
    
    # Apply list of filter on arr
    images.append(medfilt2d(arr, [3, 3]))
    images.append(medfilt2d(arr, [5, 5]))
    images.append(wiener(arr, [3, 3]))
    images.append(wiener(arr, [5, 5]))
    
    coeffs2 = pywt.dwt2(arr, 'haar')
    LL, (LH, HL, HH) = coeffs2
    
    print(LL.shape)
    print(LH.shape)
    
    images.append(LL)
    
    sv_vector = []
    # for each new image apply SVD and get SV 
    for img in images:
        s = metrics.get_SVD_s(img)
        sv_vector.append(s)
        
    sv_array = np.array(sv_vector)
    print(sv_array.shape)
    
    _, len = sv_array.shape
    
    sv_std = []
    
    # normalize each SV vectors and compute standard deviation for each sub vectors
    for i in range(len):
        sv_array[:, i] = utils.normalize_arr(sv_array[:, i])
        sv_std.append(np.std(sv_array[:, i]))
        
    
    return np.array(sv_std)


## Scenes zones data

In [8]:
# start 00020 - ref 00900 - step 10
dict_appart = {'name': 'Appart1opt02', 
               'prefix': 'appartAopt_', 
               'indices': ["00020", "00200", "00900"],
               'zones': [3, 6]}

# start 00050 - ref 01200 - step 10
dict_cuisine = {'name': 'Cuisine01', 
               'prefix': 'cuisine01_', 
               'indices': ["00050", "00400", "01200"],
               'zones': [3, 6]}

# start 00020 - ref 00950 - step 10
dict_sdb_c = {'name': 'SdbCentre', 
               'prefix': 'SdB2_', 
               'indices': ["00020", "00400", "00950"],
               'zones': [3, 6]}

# start 00020 - ref 00950 - step 10
dict_sdb_d = {'name': 'SdbDroite', 
               'prefix': 'SdB2_D_', 
               'indices': ["00020", "00400", "00950"],
               'zones': [2, 3, 10, 13]}

In [9]:
current_dict = dict_appart
interval = (30, 200)

In [10]:
images_path = compute_images_path(current_dict)
zones_data = get_images_zones(current_dict, images_path)

../fichiersSVD_light/Appart1opt02/appartAopt_00020.png
../fichiersSVD_light/Appart1opt02/appartAopt_00200.png
../fichiersSVD_light/Appart1opt02/appartAopt_00900.png


In [28]:
lab_image = metrics.get_LAB_L(zones_data[0][0])

sv_std_vector = get_sv_std(lab_image)

(100, 100)
(100, 100)
(6,)


ValueError: not enough values to unpack (expected 2, got 1)

In [14]:
indices = get_highest_values(sv_std_vector, 100)

s_lab_image = metrics.get_SVD_s(lab_image)

s_lab_image[indices]

array([4.56454151e-12, 5.98443803e-01, 2.92380551e-01, 3.11862186e+00,
       1.83881976e+00, 1.32616590e+00, 2.31162957e+00, 3.26004542e+00,
       3.69558983e+00, 4.40479644e+00, 5.86361202e+00, 4.57210378e+00,
       4.01233699e+00, 9.91121754e+00, 1.49335463e+01, 1.02598281e+01,
       9.33149634e+00, 9.02596321e+00, 7.37994999e+00, 6.44933954e+00,
       1.41221182e+01, 6.92716771e+00, 6.66692086e+00, 1.96695779e+02,
       1.35313092e+01, 4.82088913e+00, 1.30004193e+01, 1.26877063e+01,
       1.19550988e+01, 1.54599579e+01, 1.12188124e+01, 8.25570286e+00,
       8.47955315e+00, 1.20754110e+01, 7.76856416e+00, 1.55686219e+01,
       1.13219704e+01, 4.78254950e+01, 1.61024565e+01, 1.05388706e+01,
       5.00185479e+01, 1.50232416e+01, 4.66821321e+01, 5.16188334e+01,
       4.94272850e+00, 5.06946699e+01, 4.60741401e+01, 4.95388007e+01,
       1.62546781e+01, 1.70462990e+01, 1.66268801e+01, 4.64068807e+01,
       4.87707118e+01, 5.18047108e+01, 2.42465168e+01, 1.75765638e+01,
      

In [52]:
get_lowest_values(sv_std_vector, 100)

array([  1,   3,   0,  26,  28,  29,  23,  30,  27,  31,  22,  21,  20,
        24,  25,  11,  14,  18,   6,  15,  34,  17,  32,  37,  35,   2,
        12,  33,  19,  36,  38,  39,  41,  40,   7,  47,  44,  42,  43,
        45,  49,  46,  13,  48,   9,  50,  16,  10,   8,  53,  54,  55,
        51,  56,  52,  61,  57,  58,  59,  60,  63,  62,  66,  65,  64,
        67,  69, 197,  70, 104, 103, 105,  68,  72,  73,  71, 106,   4,
       107, 108, 117, 109,  74, 111,  78, 145, 114, 123, 115, 102,  97,
       152, 144, 101, 130, 131, 129, 127, 120, 116])